# Hackaton 

Welcome to this Hackaton! 

Today we are going to explore solar energy data to understand how solar energy is depending by several variables. We are going to build several prediction models to estimate the solar energy based on the variable forecast values. We will compare the prediction models to find the best model that fit our needs. Finally we are going to use this model to detect any potential anomaly detection in the solar energy measurement system. 

In [ ]:
#import functional package for our project 
from sklearn import tree
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import Ridge,LinearRegression
import seaborn as sns
from tensorflow import keras as ks
%matplotlib inline
std_figsize = (16,10)

## 1. Data exploration

In [ ]:
#import data from csv file 
data = pd.read_csv('https://raw.githubusercontent.com/peppusnight/just_for_data/main/DS.csv')
#filter only relevant data with energy not null 
data = data.loc[data.ISDAY.values>0.5,:]

# Feature selection
features = ['Radiation','HumRad','CloudRad','CLOUDCOVER','HUMIDITY','VISIBILITY',
            'TEMPERATURE','WINDSPEED','SOLARANGLE','HOUR']
# Target variable
target = 'Energy'

# Select data for training (df_X) and target variable (df_y)
data = data.loc[:, features + [target]]
df_X = data.loc[:, features]
df_y = data['Energy']
data

In [ ]:
#Data description 
data.describe()

In [ ]:
# how our target looks like? 
sns.relplot(data=data, y= 'Energy', x='Radiation', hue='HOUR')
fig = plt.gcf()
fig.set_size_inches(std_figsize)

In [ ]:
# get some insight on the data plotting some histogram, 
# check the unit of measure and value range and distribution of each variable 
_ = data.hist(figsize=std_figsize)

### 1.1 Correlation Analysis 

Evalaute the possible correlation between our target and the variables

In [ ]:
# some more closer look on Energy correlated variable 
f, ax = plt.subplots(2,2,sharey=True, figsize = std_figsize)
ax[0,0].plot(df_X['Radiation'], df_y, '.')
ax[0,0].set_xlabel('Radiation')
ax[0,1].plot(df_X['VISIBILITY'], df_y, '.')
ax[0,1].set_xlabel('VISIBILITY')
ax[1,1].plot(df_X['TEMPERATURE'], df_y, '.')
ax[1,1].set_xlabel('TEMPERATURE')
ax[1,0].plot(df_X['SOLARANGLE'], df_y, '.')
ax[1,0].set_xlabel('SOLARANGLE')
for a in ax.ravel(): a.grid(True); a.set_ylabel('Energy')

In [ ]:
# let's get perception on correlation through heat map of the variables
plt.figure(figsize=std_figsize)
sns.heatmap(data.corr(), annot=True)

## 2. Data Preprocessing

Evaluate if the data need to be normalized. It might happen that due to the different nature of the data, the values could be very different between each other and this could influence the coefficient of the regression. 

In [ ]:
# Training and test set split
features = ['Radiation','CloudRad','TEMPERATURE','WINDSPEED','HOUR']
X_train, X_test, y_train, y_test = train_test_split(df_X[features].values, df_y.values, test_size=0.3, random_state=2021)

# data normalization 
X_train_scaled = (X_train-np.mean(X_train,axis=0))/np.std(X_train,axis=0)
X_test_scaled  = (X_test-np.mean(X_train,axis=0))/np.std(X_train,axis=0)
y_train_scaled = (y_train-np.mean(y_train,axis=0))/np.std(y_train,axis=0)
y_test_scaled  = (y_test-np.mean(y_train,axis=0))/np.std(y_train,axis=0)
print('X_train number of points: ', len(X_train), 'which means ', len(X_train)/len(df_X[features])*100, '% of overall data')
print('X_test number of points: ', len(X_test), 'which means ', len(X_test)/len(df_X[features])*100, '% of overall data')

## 3. Models

in this section we are going to see what kind of different model we can elaborate to predict the solar energy as function of several variables. Check what are the feature of each models with its pros and cons. 

In [ ]:
#define Root Mean Square Error to evalaute the performance of your model 
df = pd.DataFrame(data = {'rmse_train': [],
                                'mape_train': [],
                                'rmse_test': [],
                                'mape_test': []}, index = []).T

def mean_absolute_percentage_error(y_true,y_pred):
    abs_diff = np.abs(y_true-y_pred)/y_true
    mape = np.mean(abs_diff)
    return mape

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true,y_pred))

def plot_results(y_train, y_test, X_train, X_test, model):
    f,ax = plt.subplots(2,2,sharex='col', figsize=std_figsize)
    ax[0,0].plot(y_train, '.-', label='Energy')
    ax[0,0].plot(model.predict(X_train), '.-', label='Predicted')
    ax[1,0].plot(y_train - model.predict(X_train).ravel(),'.-', label='Residuals')
    ax[0,1].plot(y_test, '.-', label='Energy')
    ax[0,1].plot(model.predict(X_test), '.-', label='Predicted')
    ax[1,1].plot(y_test - model.predict(X_test).ravel(),'.-', label='Residuals')
    ax[0,0].set_title('Training set')
    ax[0,1].set_title('Test set')
    for a in ax.ravel(): a.grid(True); a.legend(frameon=False);
    plt.figure(figsize=std_figsize)
    plt.scatter(y_test, model.predict(X_test).ravel())
    plt.plot(y_test,y_test)
    plt.xlabel('y_test')
    plt.ylabel('predicted')
    plt.grid()
    plt.show()
    rmse_train = rmse(y_train, model.predict(X_train))
    mape_train = 100*mean_absolute_percentage_error(y_train, model.predict(X_train).ravel())
    rmse_test  = rmse(y_test, model.predict(X_test))
    mape_test  = 100*mean_absolute_percentage_error(y_test, model.predict(X_test).ravel())
    print('RMSE on train set = {:.2f}'.format(rmse_train))
    print('MAPE on train set = {:.2f} '.format(mape_train))
    print('RMSE on test set = {:.2f}'.format(rmse_test))
    print('MAPE on test set = {:.2f} '.format(mape_test))
    
    return rmse_train, mape_train, rmse_test,mape_test

## Linear Regression Model

In [ ]:
##############################
# Modify only this parameter!!!!!
alpha  = 0 # Regularization parameter from 0 to Inf
solver = 'auto' # Solver:  lbfgs, svd, cholesky, lsqr, sparse_cg
tol = 0.0001 # Tolerance for convergence
##############################

# Fit the model
model_linear = Ridge(alpha = alpha, tol=tol, solver=solver)
model_linear.fit(X_train_scaled,y_train)
# print(model_linear.coef_)
# #evaluate linear model
# model_linear.predict(X_test_scaled)

df['RidgeLinear'] = plot_results(y_train, y_test, X_train_scaled, X_test_scaled, model_linear)
df

## Polynomial Regression Model

we can leverage on Explicif feature mapping to bring our ourginal features into an higher dimensional space. We can map linear features into a space with a polynomial basis, so by fitting a linear model in k feature we are really fitting a polynomial to the data.

In [ ]:
# Fit Polynomial Regression Model
######################
# Modify only this parameter!!!!!
polynomial_order = 2 # no more than 4!!!!
alpha  = 0 # Regularization parameter from 0 to Inf
solver = 'auto' # Solver: lbfgs, svd, cholesky, lsqr, sparse_cg
tol = 0.0001 # Tolerance for convergence
######################

model_polfeat = PolynomialFeatures(polynomial_order) # specify the degree of polynomial feature
model_polfeat.fit(X_train_scaled)

print("Model Feature: ",model_polfeat.get_params())
print("Model Structure:", model_polfeat.get_feature_names())

X_train_poly = model_polfeat.fit_transform(X_train_scaled)
X_test_poly = model_polfeat.fit_transform(X_test_scaled)

#fit the model on training dataset 
model_lin_polfeat = Ridge(alpha = alpha, tol=tol, solver=solver)
model_lin_polfeat.fit(X_train_poly,y_train)

#evaluate PolynomialFeatures model on test dataset
model_lin_polfeat.predict(X_test_poly)

df['RidgePoly'] = plot_results(y_train, y_test, X_train_poly, X_test_poly, model_lin_polfeat)
df

## Decision Tree

In [ ]:
################################
# Modify only these parameters!!!!
max_depth = 2
min_samples_leaf = 1
ccp_alpha = 0.0
################################

model_DT = DecisionTreeRegressor(random_state=2021, 
                                 max_depth=max_depth, 
                                 min_samples_leaf = min_samples_leaf,
                                 ccp_alpha=ccp_alpha)
model_DT.fit(X_train, y_train)

fig = plt.figure(figsize=(20,20))
_ = tree.plot_tree(model_DT,
                   feature_names=features,
                   class_names=True,
                   filled=True)

df['DecisionTree'] =plot_results(y_train, y_test, X_train, X_test, model_DT)
df

## Neural Network

In [ ]:
# NN model creation
#####################
# Modify only these parameters!!!
hidden_activation_function = 'relu' #options: relu softsign tanh
hidden_layer_1 = 10
hidden_layer_2 = 10
hidden_layer_3 = 10
epoch = 500
#####################

input_data_shape = (len(features),)
dnn_model = ks.models.Sequential()
dnn_model.add(ks.layers.Input(shape=input_data_shape, name = 'Input_Layer'))
dnn_model.add(ks.layers.Dense(hidden_layer_1, activation=hidden_activation_function, name = 'Hidden_Layer_1'))
dnn_model.add(ks.layers.Dense(hidden_layer_2, activation=hidden_activation_function, name = 'Hidden_Layer_2'))
dnn_model.add(ks.layers.Dense(hidden_layer_3, activation=hidden_activation_function, name = 'Hidden_Layer_3'))
dnn_model.add(ks.layers.Dense(1, name = 'Output_Layer'))
dnn_model.summary()

# NN Model Trainig
optimizer = 'adam'
loss_function = 'mean_absolute_error'
metric =['accuracy']
dnn_model.compile(optimizer = optimizer, loss= loss_function, metrics=metric)
history = dnn_model.fit(X_train_scaled, y_train, epochs = epoch, verbose=0)

df['NeuralNet'] = plot_results(y_train, y_test, X_train_scaled, X_test_scaled, dnn_model)
df

## Model Comparison

In [ ]:
df

## Anomaly Detection Model 

Based on your best model selected, detect a possible anomaly on a new dataset. What your model says? what your data are saying? 

In [ ]:
##############################
# Use only the following codes!!! # NN, DecisionTree, Linear, PolyLinear
model_selected = 'PolyLinear'
##############################

#import data from csv file 
data_AnomalDetect = pd.read_csv('https://raw.githubusercontent.com/peppusnight/just_for_data/main/DS_Anomaly_Version.csv')

#selection of input values and measured value in field 
features = ['Radiation','CloudRad','TEMPERATURE','WINDSPEED','HOUR']

X_model = data_AnomalDetect.loc[:, features]
Y_meas = data_AnomalDetect['Energy']

# data normalization to feed the model (normaliation according to training set on which model has been developed)
X_model_scaled = (X_model-np.mean(X_train,axis=0))/np.std(X_train,axis=0)
X_model_scaled_poly = model_polfeat.fit_transform(X_model_scaled)

if model_selected=='NN':
    Y_model = dnn_model.predict(X_model_scaled).ravel()
elif model_selected=='DecisionTree':
    Y_model = model_DT.predict(X_model)
elif model_selected=='Linear':
    Y_model = model_linear.predict(X_model_scaled)
elif model_selected=='PolyLinear':
    Y_model = model_lin_polfeat.predict(X_model_scaled_poly)

Y_model[Y_model<0] = 0
# Alm Threshold
AlmThreshold = 100
AlmThreshold = np.ones(len(data_AnomalDetect), dtype=float) * AlmThreshold

alm = pd.DataFrame()

#check for every point if they are in alarm
for i in range(len(data_AnomalDetect)):
    if abs(Y_meas[i]-Y_model[i])<100:
        alm.loc[i,'Allarm']=0
    else:
        alm.loc[i,'Allarm']=100
f,ax = plt.subplots(3,sharex='col', figsize=std_figsize)
ax[0].plot(data_AnomalDetect.index, Y_meas , label='Energy')
ax[0].plot(data_AnomalDetect.index, Y_model, label='Predicted')

ax[1].plot(data_AnomalDetect.index,Y_meas - Y_model, label='Residuals')
ax[1].plot(data_AnomalDetect.index, AlmThreshold, '--', label='Measured', color='r')
ax[1].plot(data_AnomalDetect.index, -AlmThreshold, '--', label='Measured', color='r')

ax[2].plot(data_AnomalDetect.index, alm, label='Alarm Trigger', color='r',)
ax[2].set_title('Training set')
ax[2].set_title('Test set')
for a in ax.ravel(): a.grid(True); a.legend(frameon=False);

Where the alarm has been trigged? 

check the solution and see what are the data that has been artificially altered.

## Conclusion
In this session we have seen how to interpretate your data, how to test different models and tune their parameters to get the most accurate model. Then we have seen how to use the model for application in anomaly detection. 